In [1]:
import pandas as pd
import numpy as np
from string import capwords
import yaml as yl
import re

In [2]:
reference_sheet = 'ref/30k_Cheat_Sheets_01_July.xlsx'

## Create the Special Rules Repository

### Assign Special Rules in the Core Rules + Libre Astartes/Hereticus as the "Compendium"

In [3]:
# Eventually we will have Citations, but for now, nope!
#pd.read_excel(reference_sheet, sheet_name="Special Rules", header=None, skiprows=[0], usecols="A:C", names=["Name", "Description", "Citation"])

# Load in the Data from the Excel File
pd_SpecialRules = pd.read_excel(reference_sheet, sheet_name="Special Rules", header=None, skiprows=[0], usecols="A:B", names=["Name", "Description"])

# Small Formatting Change to be in Proper English Case for Clarity with Official Sources
pd_SpecialRules['Name'] = pd_SpecialRules['Name'].apply(capwords)
pd_SpecialRules['Name'] = pd_SpecialRules['Name'].replace("\(x", "(X", regex=True)

In [4]:
with open('00-Rules/SR_compendium.yml', 'w') as file:
    yl.dump({'Compendium': pd_SpecialRules.to_dict(orient='records')}, file, default_flow_style=False)

### Assign Special Rules in the Legacy Rulebooks as the "Legacy"

In [5]:
# Load in the Data from the Excel File
pd_SpecialRules = pd.read_excel(reference_sheet, sheet_name="Legacy Special Rules", header=None, skiprows=[0], usecols="A:B", names=["Name", "Description"])

# Small Formatting Change to be in Proper English Case for Clarity with Official Sources
pd_SpecialRules['Name'] = pd_SpecialRules['Name'].apply(capwords)
pd_SpecialRules['Name'] = pd_SpecialRules['Name'].replace("\(x", "(X", regex=True)
with open('00-Rules/SR_legacy.yml', 'w') as file:
    yl.dump({'Legacy': pd_SpecialRules.to_dict(orient='records')}, file, default_flow_style=False)

## Create the Unit Types Repository

In [6]:
# Note: You need to manually excise out blank/title rows. Remember that index starts at 0, not 1!
skipRows_UnitTypes = [0, 8, 9, 30, 31]

In [7]:
# Load in the Data from the Excel File
pd_UnitTypes = pd.read_excel(reference_sheet, sheet_name="Unit Types and Sub-Types", header=None, skiprows=skipRows_UnitTypes, usecols="A:B", names=["Name", "Description"])

# Small Formatting Change to be in Proper English Case for Clarity with Official Sources
pd_UnitTypes['Name'] = pd_UnitTypes['Name'].apply(capwords)
pd_UnitTypes['Name'] = pd_UnitTypes['Name'].replace("\(x", "(X", regex=True)

In [8]:
# Remove Bullet Points for Easier Parsing into YAML Format: 
for i, UnitType_Desc in enumerate(pd_UnitTypes['Description']):
    SplitDesc = UnitType_Desc.split("\n")
    if len(SplitDesc) > 1 and SplitDesc[0][0] != '•':
        SplitDesc.remove(SplitDesc[0])
    for j, Line in enumerate(SplitDesc):
        SplitDesc[j] = Line.split('• ')[-1]
    pd_UnitTypes['Description'][i] = '\n '.join(SplitDesc)

In [9]:
with open('00-Rules/UnitTypes.yml', 'w') as file:
    yl.dump({'Legacy': pd_UnitTypes.to_dict(orient='records')}, file, default_flow_style=False)

## Create the Weapons Repository

### Start with the Close Combat Weapons

In [10]:
# You need to manually excise out blank/title rows! 
# NOTE: These are the ROW NUMBERS AS IN EXCEL!
Rows_To_Skip_Excel = [34,35,36,82,83,84,86]

# Set the Location of the Header Row
Header_Row_In_Excel = 2
pd_HeaderRow = Header_Row_In_Excel - 1

skipRows_CCWProfiles = [x - 1 for x in Rows_To_Skip_Excel]

In [11]:
pd_CCWProfiles = pd.read_excel(reference_sheet, sheet_name="CC Weapons", header=pd_HeaderRow, skiprows=skipRows_CCWProfiles)

In [12]:
# Add in the 'Range' Characteristic to be Consistent Across all Weapon Profiles
pd_CCWProfiles['Range'] = ['Combat' for i in range(len(pd_CCWProfiles['Legion']))]

for i, name in enumerate(pd_CCWProfiles['Weapon Name']):
    if type(name) != type(""):
        print(name, pd_CCWProfiles.loc[[i]])

# Ensure All Weapon Names Are in Proper English Capatilization
pd_CCWProfiles['Weapon Name'] = pd_CCWProfiles['Weapon Name'].apply(capwords)

# Fix the 'Legion' Keyword to be Self-Consistent
pd_CCWProfiles.rename(columns={"Legion":'LEGION'}, inplace=True)

# Replace the '-' Character in all Other Areas with 'None' to Prevent Parsing Issues
for key in pd_CCWProfiles.keys():
    pd_CCWProfiles[key] = pd_CCWProfiles[key].replace("-", "None", regex=False) 

# Deal with Possible Capitalization Issues in Legion Abbreviations (EX: SoH vs SOH)
pd_CCWProfiles['LEGION'] = pd_CCWProfiles['LEGION'].str.upper()

In [13]:
# Group by the Legion Column for Quicker Memory Loads
pd_GroupByLegion = pd_CCWProfiles.groupby(['LEGION'])
# For Each Legion, Do ...
for key in pd_GroupByLegion.groups.keys():
    # Convert it to a Python Dictionary for Parsing before YAML Storage
    dict_GroupByLegionKey = pd_GroupByLegion.get_group(key).to_dict(orient='records')
    # Remove the 'LEGION' Column as this will be the Parent Key for the File
    for entry in dict_GroupByLegionKey:
        entry.pop('LEGION', None)
    # Write out a BasedProfiles_{$Key}.yml File for the Given Legion
    with open('01-Weapons/Combat_'+key+'.yml', 'w') as file:
        yl.dump({key: dict_GroupByLegionKey}, file, default_flow_style=False)

### Now, Create the Ranged Weapon Profiles

In [14]:
# You need to manually excise out blank/title rows! 
# NOTE: These are the ROW NUMBERS AS IN EXCEL!
Rows_To_Skip_Excel = [139, 149, 150, 211, 212, 213, 240, 241]

# Set the Location of the Header Row
Header_Row_In_Excel = 1
pd_HeaderRow = Header_Row_In_Excel - 1

skipRows_RangedProfiles = [x - 1 for x in Rows_To_Skip_Excel]

In [15]:
pd_RangedProfiles = pd.read_excel(reference_sheet, sheet_name="Ranged Weapons", header=pd_HeaderRow, skiprows=skipRows_RangedProfiles)

In [16]:
# Fix the 'Legion' Keyword to be Self-Consistent
pd_RangedProfiles.rename(columns={"Legion":'LEGION'}, inplace=True)

# Replace the '-' Character in all Other Areas with 'None' to Prevent Parsing Issues
for key in pd_RangedProfiles.keys():
    pd_RangedProfiles[key] = pd_RangedProfiles[key].replace("-", "None", regex=False) 

# Deal with Possible Capitalization Issues in Legion Abbreviations (EX: SoH vs SOH)
pd_RangedProfiles['LEGION'] = pd_RangedProfiles['LEGION'].str.upper()

In [17]:
# Address the Issue of Multiple Profile Weapons

# Solution: Make each profile its own "weapon" row in Pandas and remove the parent.

max_rows = len(pd_RangedProfiles['Weapon Name'])
different_range_names = ['Close', 'Medium', 'Far'] # For Conversion Beamer Cause its Special
for i in np.where(pd.isnull(pd_RangedProfiles['Str']))[0]:
    # Set the Parent Weapon's Name
    parent_weapon_name = pd_RangedProfiles['Weapon Name'][i]
    # Set a Search Cap for Secondary Profiles & Ensure it Doesn't Exceed the Table's Length
    search_cap = 5
    if i+search_cap > max_rows: search_cap = max_rows - i
    for j in range(search_cap):
        # Set the Current Weapon Profile Name
        current_profile = pd_RangedProfiles['Weapon Name'][i+j]
        # Check to Make Sure the Parent & Current Weapon Profile Names aren't Secondaries
        if '(Secondary)' in parent_weapon_name and '(Secondary)' in current_profile:
            continue
        if type(current_profile) == type(''):
            # Check for the Auxillary Profile Delimeter (Leading '-' in the Name)
            if '-' in current_profile[0:4]:
                current_profile = current_profile.replace('-',' ')
                current_profile = current_profile.replace('(','')
                current_profile = current_profile.replace(')','')
                parent_weapon_name = parent_weapon_name.replace('-',' ')
                current_profile = re.sub(' +', ' ', current_profile)
                parent_weapon_name = parent_weapon_name.strip()
                #print(i+j, parent_weapon_name+" ~"+current_profile)
                pd_RangedProfiles.at[i+j, 'Weapon Name'] = parent_weapon_name+" ~"+current_profile
        else:
            #print(i+j, parent_weapon_name, "~", different_range_names[j-1])
            pd_RangedProfiles.at[i+j, 'Weapon Name'] = parent_weapon_name+" ~ "+different_range_names[j-1]
pd_RangedProfiles.drop(np.where(pd.isnull(pd_RangedProfiles['Str']))[0], inplace=True)

In [18]:
# Ensure All Weapon Names Are in Proper English Capatilization
pd_RangedProfiles['Weapon Name'] = pd_RangedProfiles['Weapon Name'].apply(capwords)

# Ensure that the Weird Whirlwind Launcher Names Are Fixed
pd_RangedProfiles = pd_RangedProfiles.replace(" He ", " HE ", regex=True)
pd_RangedProfiles = pd_RangedProfiles.replace(" Ap ", " AP ", regex=True)
pd_RangedProfiles = pd_RangedProfiles.replace(" Aa ", " AA ", regex=True)

In [19]:
# Deal with the Iliastus pattern assault cannon Issue of "IF/BA"
ls_LegionNames = ['BA', 'IF']
row_IF_IPAC = pd_RangedProfiles.loc[pd_RangedProfiles['LEGION'] == 'IF/BA'].copy(deep=True)
pd_RangedProfiles = pd_RangedProfiles.append(row_IF_IPAC, ignore_index=True)
index_for_rows = pd_RangedProfiles.index[pd_RangedProfiles['LEGION']=='IF/BA'].tolist()
for i, index in enumerate(index_for_rows):
    pd_RangedProfiles.at[index, 'LEGION'] = ls_LegionNames[i]

In [20]:
# Group by the Legion Column for Quicker Memory Loads
pd_GroupByLegion = pd_RangedProfiles.groupby(['LEGION'])
# For Each Legion, Do ...
for key in pd_GroupByLegion.groups.keys():
    # Convert it to a Python Dictionary for Parsing before YAML Storage
    dict_GroupByLegionKey = pd_GroupByLegion.get_group(key).to_dict(orient='records')
    # Remove the 'LEGION' Column as this will be the Parent Key for the File
    for entry in dict_GroupByLegionKey:
        entry.pop('LEGION', None)
    # Write out a BasedProfiles_{$Key}.yml File for the Given Legion
    with open('01-Weapons/Ranged_'+key+'.yml', 'w') as file:
        yl.dump({key: dict_GroupByLegionKey}, file, default_flow_style=False)

## Create the Profiles Repository 

### Start with All of the Models that are on Bases (Referred to as 'BasedProfiles')

In [21]:
# You need to manually excise out blank/title rows! 
# NOTE: These are the ROW NUMBERS AS IN EXCEL!
Rows_To_Skip_Excel = [53,54,55,157,158,159,160,168,169]

# Set the Location of the Header Row
Header_Row_In_Excel = 4
pd_HeaderRow = Header_Row_In_Excel - 1

skipRows_BasedProfiles = [x - 1 for x in Rows_To_Skip_Excel]

In [22]:
# Load in the Data from the Excel File
pd_BasedProfiles = pd.read_excel(reference_sheet, sheet_name="Unit Profiles", header=pd_HeaderRow, skiprows=skipRows_BasedProfiles)

# Deal with Possible Capitalization Issues in Legion Abbreviations (EX: SoH vs SOH)
pd_BasedProfiles['LEGION'] = pd_BasedProfiles['LEGION'].str.upper()

# Deal with TROOP vs TROOPS (Prefer the Latter, as per rulebook)
pd_BasedProfiles['FOC Slot'] = pd_BasedProfiles['FOC Slot'].replace("TROOP", "TROOPS", regex=False)

# Replace the '-' Character with 'ATTACHED' Keyword
pd_BasedProfiles['FOC Slot'] = pd_BasedProfiles['FOC Slot'].replace("-", "ATTACHED", regex=False)

# Replace the '-' Character in all Other Areas with 'None' to Prevent Parsing Issues
for key in pd_BasedProfiles.keys():
    pd_BasedProfiles[key] = pd_BasedProfiles[key].replace("-", "None", regex=False) 

In [23]:
# Group by the Legion Column for Quicker Memory Loads
pd_GroupByLegion = pd_BasedProfiles.groupby(['LEGION'])

In [24]:
# For Each Legion, Do ...
for key in pd_GroupByLegion.groups.keys():
    # Convert it to a Python Dictionary for Parsing before YAML Storage
    dict_GroupByLegionKey = pd_GroupByLegion.get_group(key).to_dict(orient='records')
    # Remove the 'LEGION' Column as this will be the Parent Key for the File
    for entry in dict_GroupByLegionKey:
        entry.pop('LEGION', None)
    # Write out a BasedProfiles_{$Key}.yml File for the Given Legion
    with open('02-Profiles/BasedProfiles_'+key+'.yml', 'w') as file:
        yl.dump({key: dict_GroupByLegionKey}, file, default_flow_style=False)

### Now Tackle All of the Vechicles

In [25]:
# You need to manually excise out blank/title rows! 
# NOTE: These are the ROW NUMBERS AS IN EXCEL!
Rows_To_Skip_Excel = [35,36,37,38]

# Set the Location of the Header Row
Header_Row_In_Excel = 1
pd_HeaderRow = Header_Row_In_Excel - 1

skipRows_VehicleProfiles = [x - 1 for x in Rows_To_Skip_Excel]

In [26]:
# Load in the Data from the Excel File
pd_VehicleProfiles = pd.read_excel(reference_sheet, sheet_name="Vehicle Profiles", header=pd_HeaderRow, skiprows=skipRows_VehicleProfiles)

In [27]:
# Deal with Possible Capitalization Issues in Legion Abbreviations (EX: SoH vs SOH)
pd_BasedProfiles['LEGION'] = pd_BasedProfiles['LEGION'].str.upper()

# Replace the '-' Character in all Other Areas with 'None' to Prevent Parsing Issues
for key in pd_VehicleProfiles.keys():
    pd_VehicleProfiles[key] = pd_VehicleProfiles[key].replace("-", "None", regex=False)
pd_VehicleProfiles.rename(columns={"TRANSPORT ":'Transport'}, inplace=True)

In [29]:
# Group by the Legion Column for Quicker Memory Loads
pd_GroupByLegion = pd_VehicleProfiles.groupby(['LEGION'])

In [30]:
# For Each Legion, Do ...
for key in pd_GroupByLegion.groups.keys():
    # Convert it to a Python Dictionary for Parsing before YAML Storage
    dict_GroupByLegionKey = pd_GroupByLegion.get_group(key).to_dict(orient='records')
    # Remove the 'LEGION' Column as this will be the Parent Key for the File
    for entry in dict_GroupByLegionKey:
        entry.pop('LEGION', None)
    # Write out a BasedProfiles_{$Key}.yml File for the Given Legion
    with open('02-Profiles/VehicleProfiles_'+key+'.yml', 'w') as file:
        yl.dump({key: dict_GroupByLegionKey}, file, default_flow_style=False)

### Finally the Fortifications

In [31]:
# Skip the Fortress of Redeption Line (To Be Dealt  With Later ...)
skipRows_FortProfiles = [5]

In [32]:
# Load in the Data from the Excel File
pd_FortProfiles = pd.read_excel(reference_sheet, sheet_name="Fortifications", header=0, skiprows=skipRows_FortProfiles)

In [33]:
# Deal with Possible Capitalization Issues in Legion Abbreviations (EX: SoH vs SOH)
pd_BasedProfiles['LEGION'] = pd_BasedProfiles['LEGION'].str.upper()

# Replace the '-' Character in all Other Areas with 'None' to Prevent Parsing Issues
for key in pd_FortProfiles.keys():
    pd_FortProfiles[key] = pd_FortProfiles[key].replace("-", "None", regex=False)
pd_FortProfiles.rename(columns={"TRANSPORT ":'Transport'}, inplace=True)

In [34]:
# Deal with the Fortress of Redemption Issue
pd_FortProfiles.at[4, 'Unit Name'] = "Fortress of Redemption Bunker Annex"
pd_FortProfiles.at[5, 'Unit Name'] = "Fortress of Redemption Tower"

In [35]:
# Group by the Legion Column for Quicker Memory Loads
pd_GroupByLegion = pd_FortProfiles.groupby(['LEGION'])

In [36]:
# For Each Legion, Do ...
for key in pd_GroupByLegion.groups.keys():
    # Convert it to a Python Dictionary for Parsing before YAML Storage
    dict_GroupByLegionKey = pd_GroupByLegion.get_group(key).to_dict(orient='records')
    # Remove the 'LEGION' Column as this will be the Parent Key for the File
    for entry in dict_GroupByLegionKey:
        entry.pop('LEGION', None)
    # Write out a BasedProfiles_{$Key}.yml File for the Given Legion
    with open('02-Profiles/FortificationProfiles_'+key+'.yml', 'w') as file:
        yl.dump({key: dict_GroupByLegionKey}, file, default_flow_style=False)